this notebook is used to calculate how much energy goes into which state

In [ ]:
from postprocessing import *

In [ ]:
mpl.rcParams['figure.dpi']= 200

In [ ]:
density = 20
mach = 4
jetpower = 43
start = '15'

In [ ]:
simulation_directory = str(f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_jet_d{density}_m{mach}/jet{jetpower}_{start}')
output_directory = simulation_directory + "/output/"
figures_directory = simulation_directory + "/output/figures/"

## total energy comparison

In [ ]:
i_file = 20
filename = "snap_%03d.hdf5" % (i_file)
snap_data = h5py.File(output_directory + filename, "r")

In [ ]:
thermal_energy = (snap_data['PartType0/InternalEnergy'][:] * 
                  snap_data['PartType0/Masses'][:] * snap_data['PartType0/Jet_Tracer'][:])

In [ ]:
velocity = np.sqrt(snap_data['PartType0/Velocities'][:][:,0] ** 2 +
                   snap_data['PartType0/Velocities'][:][:,1] ** 2 +
                   snap_data['PartType0/Velocities'][:][:,2] ** 2)
kinetic_energy = velocity ** 2 * snap_data['PartType0/Masses'][:] * snap_data['PartType0/Jet_Tracer'][:] / 2

In [ ]:
np.sum(thermal_energy + kinetic_energy) * unit_velocity ** 2 * unit_mass

In [ ]:
10 ** jetpower * get_time_from_snap(snap_data) * unit_time

## energy coupling in time

In [ ]:
i_file = -1
result = []

while True:
    i_file += 1
    filename = "snap_%03d.hdf5" % (i_file)
    try:
        snap_data = h5py.File(output_directory + filename, "r")
    except:
        break
    time = get_time_from_snap(snap_data)
    
    jet_tracer = np.zeros_like(snap_data['PartType0/Jet_Tracer'][:])
    jet_no_tracer = np.zeros_like(snap_data['PartType0/Jet_Tracer'][:])
    snap_data['PartType0/Jet_Tracer'][:][snap_data['PartType0/Jet_Tracer'][:] < 0] = 0
    x, y, z = snap_data['PartType0/Coordinates'][:].T
    radius = np.sqrt((x - 1000) ** 2 + (y - 1000) ** 2 + (z - 1000) ** 2)
    
    mask_jet = (snap_data['PartType0/Jet_Tracer'][:] > 1e-3) #& (radius < 500)
    mask_no_jet = (snap_data['PartType0/Jet_Tracer'][:] < 1e-3) #& (radius < 500)
    
    jet_tracer[mask_jet] = snap_data['PartType0/Jet_Tracer'][:][mask_jet]
    jet_no_tracer[mask_no_jet] = 1 - snap_data['PartType0/Jet_Tracer'][:][mask_no_jet]
    
    thermal_energy = snap_data['PartType0/InternalEnergy'][:] * snap_data['PartType0/Masses'][:] #* jet_no_tracer
    velocity = np.sqrt(snap_data['PartType0/Velocities'][:][:,0] ** 2 +
                       snap_data['PartType0/Velocities'][:][:,1] ** 2 +
                       snap_data['PartType0/Velocities'][:][:,2] ** 2)
    kinetic_energy = velocity ** 2 * snap_data['PartType0/Masses'][:] / 2 #* jet_no_tracer
    result.append([get_time_from_snap(snap_data) * unit_time_in_megayr,
                   np.sum(thermal_energy) * unit_velocity ** 2 * unit_mass,
                   np.sum(kinetic_energy) * unit_velocity ** 2 * unit_mass,
                   10 ** jetpower * get_time_from_snap(snap_data) * unit_time])

In [ ]:
result = np.array(result).T

In [ ]:
fig, ax = plt.subplots(figsize=(7,4))
ax.plot(result[0], result[3], label='input energy', c='black')
ax.plot(result[0], result[1] + result[2], label='total energy in medium', c='tab:blue')
ax.plot(result[0], result[1], label='internal energy', c='tab:red')
ax.plot(result[0], result[2], label='kinetic energy', c='tab:green')

plt.yscale('log')
plt.legend(loc='lower right', framealpha=0.7) # loc=(0.65, 0.65)
ax.set_ylim(1e55, 5e57)
ax.set_xlim(0, 0.8)
ax.set_xlabel('t [Myr]', fontsize=15)
plt.grid(ls='--', zorder = 100, alpha=0.5)
ax.set_ylabel('energy [erg]', fontsize=15)

## turbulent box vs box with the jet

In [ ]:
density = 100
mach = 4
jetpower = 44
start = '12'

In [ ]:
simulation_directory_jet = str(f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_drive_center_d{density}_m{mach}/jet{jetpower}_{start}')
output_directory_jet = simulation_directory_jet + "/output/"
figures_directory_jet = simulation_directory_jet + "/output/figures/"

simulation_directory_turb = str(f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_drive_center_d{density}_m{mach}/turb_alter_{start}')
output_directory_turb = simulation_directory_turb + "/output/"
figures_directory_turb = simulation_directory_turb + "/output/figures/"

In [ ]:
i_file_jet = -1
result_jet = []

while True:
    i_file_jet += 1
    filename = "snap_%03d.hdf5" % (i_file_jet)
    try:
        snap_data = h5py.File(output_directory_jet + filename, "r")
    except:
        break
    time = get_time_from_snap(snap_data)
    
    jet_tracer = np.zeros_like(snap_data['PartType0/Jet_Tracer'][:])
    jet_no_tracer = np.zeros_like(snap_data['PartType0/Jet_Tracer'][:])
    snap_data['PartType0/Jet_Tracer'][:][snap_data['PartType0/Jet_Tracer'][:] < 0] = 0
    x, y, z = snap_data['PartType0/Coordinates'][:].T
    radius = np.sqrt((x - 1000) ** 2 + (y - 1000) ** 2 + (z - 1000) ** 2)
    
    mask_jet = (snap_data['PartType0/Jet_Tracer'][:] > 1e-3) #& (radius < 500)
    mask_no_jet = (snap_data['PartType0/Jet_Tracer'][:] < 1e-3) #& (radius < 500)
    
    jet_tracer[mask_jet] = snap_data['PartType0/Jet_Tracer'][:][mask_jet]
    jet_no_tracer[mask_no_jet] = 1 - snap_data['PartType0/Jet_Tracer'][:][mask_no_jet]
    
    thermal_energy = snap_data['PartType0/InternalEnergy'][:] * snap_data['PartType0/Masses'][:] #* jet_tracer
    velocity = np.sqrt(snap_data['PartType0/Velocities'][:][:,0] ** 2 +
                       snap_data['PartType0/Velocities'][:][:,1] ** 2 +
                       snap_data['PartType0/Velocities'][:][:,2] ** 2)
    kinetic_energy = velocity ** 2 * snap_data['PartType0/Masses'][:] / 2 #* jet_tracer
    result_jet.append([get_time_from_snap(snap_data) * unit_time_in_megayr,
                   np.sum(thermal_energy) * unit_velocity ** 2 * unit_mass,
                   np.sum(kinetic_energy) * unit_velocity ** 2 * unit_mass,
                   10 ** jetpower * get_time_from_snap(snap_data) * unit_time])
    
    
i_file_turb = -1
result_turb = []

while i_file_turb < i_file_jet - 1:
    i_file_turb += 1
    filename = "snap_%03d.hdf5" % (i_file_turb)
    try:
        snap_data = h5py.File(output_directory_turb + filename, "r")
    except:
        break
    time = get_time_from_snap(snap_data)
    
#     jet_tracer = np.zeros_like(snap_data['PartType0/Jet_Tracer'][:])
#     jet_no_tracer = np.zeros_like(snap_data['PartType0/Jet_Tracer'][:])
#     snap_data['PartType0/Jet_Tracer'][:][snap_data['PartType0/Jet_Tracer'][:] < 0] = 0
    x, y, z = snap_data['PartType0/Coordinates'][:].T
    radius = np.sqrt((x - 1000) ** 2 + (y - 1000) ** 2 + (z - 1000) ** 2)
    
#     mask_jet = (snap_data['PartType0/Jet_Tracer'][:] > 1e-3) #& (radius < 500)
#     mask_no_jet = (snap_data['PartType0/Jet_Tracer'][:] < 1e-3) #& (radius < 500)
    
#     jet_tracer[mask_jet] = snap_data['PartType0/Jet_Tracer'][:][mask_jet]
#     jet_no_tracer[mask_no_jet] = 1 - snap_data['PartType0/Jet_Tracer'][:][mask_no_jet]
    
    thermal_energy = snap_data['PartType0/InternalEnergy'][:] * snap_data['PartType0/Masses'][:] #* jet_tracer
    velocity = np.sqrt(snap_data['PartType0/Velocities'][:][:,0] ** 2 +
                       snap_data['PartType0/Velocities'][:][:,1] ** 2 +
                       snap_data['PartType0/Velocities'][:][:,2] ** 2)
    kinetic_energy = velocity ** 2 * snap_data['PartType0/Masses'][:] / 2 #* jet_tracer
    result_turb.append([get_time_from_snap(snap_data) * unit_time_in_megayr,
                   np.sum(thermal_energy) * unit_velocity ** 2 * unit_mass,
                   np.sum(kinetic_energy) * unit_velocity ** 2 * unit_mass,
                   10 ** jetpower * get_time_from_snap(snap_data) * unit_time])

In [ ]:
result_jet = np.array(result_jet).T
result_turb = np.array(result_turb).T

In [ ]:
result_turb[2]

In [ ]:
plt.plot(result_jet[0], result_jet[2])
plt.plot(result_turb[0] - result_turb[0][0], result_turb[2])
plt.xlabel('time [Myr]')
plt.ylabel('kinetic energy')

In [ ]:
fig, ax = plt.subplots(figsize=(7,4))
ax.plot(result_jet[0], result_jet[3], label='input energy', c='black')
ax.plot(result_jet[0], result_jet[1] - result_turb[1] + result_jet[2] - result_turb[2], label='total energy', c='tab:blue')
ax.plot(result_jet[0], result_jet[1] - result_turb[1], label='internal energy', c='tab:red')
ax.plot(result_jet[0], result_jet[2] - result_turb[2], label='kinetic energy', c='tab:green')

plt.yscale('log')
plt.legend(loc='lower right', framealpha=0.7) # loc=(0.65, 0.65)
ax.set_ylim(1e55, 5e57)
ax.set_xlim(0, 1)
ax.set_xlabel('t [Myr]', fontsize=15)
plt.grid(ls='--', zorder = 100, alpha=0.5)
ax.set_ylabel('energy difference [erg]', fontsize=15)

In [ ]:
fig, ax = plt.subplots(figsize=(7,4))

ax.plot(result_jet[0], (result_jet[1] - result_turb[1] + result_jet[2] - result_turb[2]) / (result_turb[1] + result_turb[2]),
        label='total energy', c='tab:blue')
ax.plot(result_jet[0], (result_jet[1] - result_turb[1]) / result_turb[1], label='internal energy', c='tab:red')
ax.plot(result_jet[0], (result_jet[2] - result_turb[2]) / result_turb[2], label='kinetic energy', c='tab:green')

# plt.yscale('log')
plt.legend(loc='lower right', framealpha=0.7) # loc=(0.65, 0.65)
# ax.set_ylim(-1, 1)
ax.set_xlim(0, 1)
ax.set_xlabel('t [Myr]', fontsize=15)
plt.grid(ls='--', zorder = 100, alpha=0.5)
ax.set_ylabel('relative energy difference', fontsize=15)

## cooling

In [ ]:
i_file_turb = 17

In [ ]:
filename = "snap_%03d.hdf5" % (i_file_turb)
snap_data = h5py.File(output_directory_turb + filename, "r")

In [ ]:
snap_data['PartType0'].keys()

In [ ]:
coolingrates = snap_data['PartType0/CoolingRate'][:]
mask = (coolingrates > 0)
# timescale = (snap_data['PartType0/InternalEnergy'][:][mask] * PROTONMASS ** 2) \
#             / (coolingrates[mask] * snap_data['PartType0/Density'][:][mask] * mu ** 2)
# timescale *= unit_velocity ** 2 / unit_density
timescale = snap_data['PartType0/InternalEnergy'][:][mask] / coolingrates[mask] * unit_time_in_megayr

In [ ]:
timescale.mean() # Myr

In [ ]:
y,x, _ = plt.hist(timescale, bins=np.logspace(-4, 6, 50), weights=snap_data['PartType0/Masses'][:][mask])
plt.axvline(x[y.argmax()], c='black', label=f'max: {np.round(x[y.argmax()], 2)} Myr')
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.xlim(1e-4, 1e6)
plt.xlabel('timescale [Myr]')